In [38]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from heart_categories_new import *
from generic_my_ds_utilities import *
from woe import *

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score,recall_score, precision_score 
from sklearn.metrics import f1_score, log_loss
from sklearn.cluster import KMeans
from mlencoders.target_encoder import TargetEncoder
from sklearn.neighbors import KNeighborsClassifier

trainFeaturesFile = 'HeartDisease/Warm_Up_Machine_Learning_with_a_Heart_-_Train_Values.csv'
trainLabelsFile = 'HeartDisease/Warm_Up_Machine_Learning_with_a_Heart_-_Train_Labels.csv'
testFile = 'HeartDisease/Warm_Up_Machine_Learning_with_a_Heart_-_Test_Values.csv'
                                




In [50]:
def prepareData(data, woe=None):
    
    toDrop = ['patient_id', 'resting_blood_pressure', 'serum_cholesterol_mg_per_dl', 'oldpeak_eq_st_depression',
         'age', 'max_heart_rate_achieved']
    toCateg = ['thal','slope_of_peak_exercise_st_segment', 'chest_pain_type',
       'num_major_vessels', 'fasting_blood_sugar_gt_120_mg_per_dl',
       'resting_ekg_results', 'sex', 'exercise_induced_angina', 'disc_serum_cholesterol_mg_per_dl',
       'disc_oldpeak_eq_st_depression', 'disc_age',
       'disc_max_heart_rate_achieved', 'disc_resting_blood_pressure']
    
    df = data.copy()
    df.thal = LabelEncoder().fit_transform(df.thal)
    df['disc_resting_blood_pressure'] = df.apply(get_resting_blood_pressure_categories, axis=1)
    df['disc_serum_cholesterol_mg_per_dl'] = df.apply(get_serum_cholesterol_mg_per_dl_categ, axis=1)
    df['disc_oldpeak_eq_st_depression'] = df.apply(get_oldpeak_eq_st_depression_categ, axis=1)
    df['disc_age'] = df.apply(getAgeGroup, axis=1)
    df['disc_max_heart_rate_achieved'] = df.apply(get_max_heart_rate_achieved_categories, axis=1)
    for fea in toCateg:
        df[fea] = df[fea].astype('category')
    df = df.drop(toDrop, axis=1)
    if woe is None:
        woe = getWoe(df)
        df = df.drop('heart_disease_present', axis=1)
    for feat, attributes in woe.items():
        df[feat].replace(attributes, inplace=True)

    return  df, woe
    


def prepareDataWithoutWoE(data):
    
    toDrop = ['patient_id', 'resting_blood_pressure', 'serum_cholesterol_mg_per_dl', 'oldpeak_eq_st_depression',
         'age', 'max_heart_rate_achieved']
    toCateg = ['thal','slope_of_peak_exercise_st_segment', 'chest_pain_type',
       'num_major_vessels', 'fasting_blood_sugar_gt_120_mg_per_dl',
       'resting_ekg_results', 'sex', 'exercise_induced_angina', 'disc_serum_cholesterol_mg_per_dl',
       'disc_oldpeak_eq_st_depression', 'disc_age',
       'disc_max_heart_rate_achieved', 'disc_resting_blood_pressure']
    
    df = data.copy()
    df.thal = LabelEncoder().fit_transform(df.thal)
    df['disc_resting_blood_pressure'] = df.apply(get_resting_blood_pressure_categories, axis=1)
    df['disc_serum_cholesterol_mg_per_dl'] = df.apply(get_serum_cholesterol_mg_per_dl_categ, axis=1)
    df['disc_oldpeak_eq_st_depression'] = df.apply(get_oldpeak_eq_st_depression_categ, axis=1)
    df['disc_age'] = df.apply(getAgeGroup, axis=1)
    df['disc_max_heart_rate_achieved'] = df.apply(get_max_heart_rate_achieved_categories, axis=1)
    for fea in toCateg:
        df[fea] = df[fea].astype('category')
    df = df.drop(toDrop, axis=1)
   
    return  df
    


def auc(X, y, clf):
    clf.fit(X, y)
    predictions = clf.predict_proba(X)[:,1] 
    auc = roc_auc_score(y, predictions) 
    return(auc)

clf = DecisionTreeClassifier()

In [3]:
features = pd.read_csv(trainFeaturesFile)
labels = pd.read_csv(trainLabelsFile)
df = features.merge(labels, on='patient_id')
dftestWoe, woe = prepareData(df)
dftest = prepareDataWithoutWoE(df)
X = dftest.drop('heart_disease_present', axis=1)
y = dftest['heart_disease_present']

In [4]:
#calculate AUC ROC fpr each feature
for col in X.columns:
    print(col + " = " + str(auc(X[[col]], df.heart_disease_present, clf)))

slope_of_peak_exercise_st_segment = 0.6859999999999999
thal = 0.7745
chest_pain_type = 0.7696875
num_major_vessels = 0.7223125
fasting_blood_sugar_gt_120_mg_per_dl = 0.50125
resting_ekg_results = 0.5791250000000001
sex = 0.65625
exercise_induced_angina = 0.71
disc_resting_blood_pressure = 0.53675
disc_serum_cholesterol_mg_per_dl = 0.56575
disc_oldpeak_eq_st_depression = 0.72325
disc_age = 0.615
disc_max_heart_rate_achieved = 0.716125


In [5]:
#Finding best categorization (based on AUC ROC)
def getBestCategBins(df, varialble, target, maxDiff, step):
    times = []
    dec_div = 1
    max_auc = 0
    bestComb = ()
    min_val = df[varialble].min()
    max_val = df[varialble].max()
    if maxDiff<1:
        print('yes')
        dec_div = 10
        maxDiff = int(maxDiff *10)
        step = int(step*10)
        min_val = int(min_val*10)
        max_val = int(max_val*10)
    combination_4 = [(a/dec_div, b/dec_div, c/dec_div, d/dec_div) for a in range(min_val, max_val-3*maxDiff, step) for b in range (a+maxDiff, max_val-2*maxDiff, step) for c in range (b+maxDiff, max_val-maxDiff, step) for d in range (c+maxDiff, max_val, step)]
    combination_3 = [(a/dec_div, b/dec_div, c/dec_div) for a in range(min_val, max_val-2*maxDiff, step) for b in range(a+maxDiff, max_val-maxDiff, step) for c in range(b+maxDiff, max_val, step) ]
    combination_2 = [(a/dec_div, b/dec_div) for a in range(min_val, max_val-maxDiff, step) for b in range(a+maxDiff, max_val, step) ]
    clf = LogisticRegression(solver='lbfgs')
    num_comb = getNumberOfCombinations(min_val, max_val, maxDiff, step)
    print("Number of combinations to try:" + str(num_comb))
    notreported = False
    for (a, b, c, d) in combination_4:
        if len(times) < 50:
            now = pd.Timestamp.now()
            times.append(now)
        elif notreported==False:
            times = pd.Series(times)
            notreported = True
            time = (times.diff().dropna().mean() * num_comb).seconds
            print('For this number of combinations is estimated in minutes: '+ str(time/60))
        def variable_categories(row):
            if row[varialble] < a:
                return 1
            elif row[varialble] >= a and row[varialble]< b:
                return 2
            elif row[varialble] >= b and row[varialble]< c:
                return 3
            elif row[varialble] >= c and row[varialble]< d:
                return 4
            elif row[varialble] >= d:
                return 5
        df['disc_variable'] = df.apply(variable_categories, axis=1).astype('category')
        auc1 = auc(df[['disc_variable']], df.heart_disease_present, clf)
        if auc1 > max_auc:
            max_auc = auc1
            bestComb = (a,b,c,d)
            print(max_auc,  (a, b, c,d))
    print('Done with 4')
    for (a,b, c) in combination_3:
        def variable_categories(row):
            if row[varialble] < a:
                return 1
            elif row[varialble]  >= a and row[varialble]< b:
                return 2
            elif row[varialble] >= b and row[varialble]< c:
                return 3
            elif row[varialble] >= c:
                return 4 
        df['disc_variable'] = df.apply(variable_categories, axis=1).astype('category')
        auc1 = auc(df[['disc_variable']], df[target], clf)
        if auc1 > max_auc:
            max_auc = auc1
            bestComb = (a,b,c)
            print(max_auc,  (a, b, c))
    print('Done with 3')
    
    for (a,b) in combination_2:
        def variable_categories(row):
            if row[varialble] < a:
                return 1
            elif row[varialble] >= a and row[varialble]< b:
                return 2
            elif row[varialble] >= b:
                return 3
        
        df['disc_variable'] = df.apply(variable_categories, axis=1).astype('category')
        auc1 = auc(df[['disc_variable']], df[target], clf)
        if auc1 > max_auc:
            max_auc = auc1
            bestComb = (a,b)
            print(max_auc,  (a, b))
        return max_auc, bestComb
def getNumberOfCombinations(min_val, max_val, maxDiff, step):
    combination_4 = [(a, b, c, d) for a in range(min_val, max_val-3*maxDiff, step) for b in range (a+maxDiff, max_val-2*maxDiff, step) for c in range (b+maxDiff, max_val-maxDiff, step) for d in range (c+maxDiff, max_val, step)]
    combination_3 = [(a, b, c) for a in range(min_val, max_val-2*maxDiff, step) for b in range(a+maxDiff, max_val-maxDiff, step) for c in range(b+50, max_val, step) ]
    combination_2 = [(a, b) for a in range(min_val, max_val-maxDiff, step) for b in range(a+maxDiff, max_val, step) ]
    combination = combination_4 + combination_3 + combination_2
    return(len(combination_2) + len (combination_3) + len(combination_4))

In [ ]:
#serum_cholesterol_mg_per_dl 0.60175 (126, 146, 216, 272)
getBestCategBins(df, 'serum_cholesterol_mg_per_dl', 'heart_disease_present', 30, 20)

In [ ]:
getBestCategBins(df1, 'max_heart_rate_achieved', 'heart_disease_present', 5, 2) #110min

In [ ]:
getBestCategBins(df1, 'age', 'heart_disease_present', 2, 1) #76min

In [ ]:
getBestCategBins(df1, 'resting_blood_pressure', 'heart_disease_present', 4, 2) #46min

In [ ]:
getBestCategBins(df1, 'oldpeak_eq_st_depression', 'heart_disease_present', 0.2, 0.1) #147min

In [98]:
#Bulding and testing polynomial features
pf = PolynomialFeatures(degree=2, interaction_only=True)
new = pf.fit_transform(dftest.drop('heart_disease_present', axis=1))
newdf = pd.DataFrame(new, columns=pf.get_feature_names())
newdf = pd.concat([newdf, y], axis=1)
newdf.head()

#extracting the features with best auc (>0.8) - resulting dataframe : impFea_df
aucs = {}
allaucs = {}
for col in newdf.drop('heart_disease_present', axis=1).columns:
    b = auc(newdf[[col]], newdf.heart_disease_present, clf)
    allaucs[col] = b

In [103]:
for fea, aucFea in allaucs.items():
    comps = fea.split()
    if len(comps) > 1:
        if allaucs[comps[0]] < aucFea and allaucs[comps[1]] < aucFea:
            aucs[fea] = aucFea
aucs

{'x5': 0.7458125,
 'x0 x1': 0.8076875000000001,
 'x0 x2': 0.8154374999999999,
 'x0 x3': 0.7475625,
 'x0 x6': 0.749875,
 'x0 x7': 0.7273125,
 'x0 x8': 0.723,
 'x0 x9': 0.7063125,
 'x0 x10': 0.7506250000000001,
 'x0 x11': 0.7184375000000001,
 'x1 x2': 0.85725,
 'x1 x8': 0.7881250000000001,
 'x1 x10': 0.83375,
 'x1 x11': 0.80175,
 'x2 x6': 0.7789375000000001,
 'x2 x9': 0.7717499999999999,
 'x2 x10': 0.853125,
 'x2 x11': 0.7776875,
 'x3 x6': 0.7287500000000001,
 'x3 x8': 0.738,
 'x3 x9': 0.7230624999999999,
 'x3 x10': 0.7461875,
 'x3 x11': 0.725,
 'x3 x12': 0.740125,
 'x4 x8': 0.5523750000000001,
 'x5 x8': 0.5876875000000001,
 'x5 x9': 0.6024375000000001,
 'x5 x11': 0.6249375000000001,
 'x6 x8': 0.6761874999999999,
 'x6 x9': 0.7063125,
 'x6 x10': 0.76125,
 'x6 x11': 0.723375,
 'x6 x12': 0.7645625,
 'x7 x8': 0.7209375,
 'x7 x9': 0.720875,
 'x7 x10': 0.731,
 'x7 x11': 0.7161875,
 'x7 x12': 0.723125,
 'x8 x9': 0.6300625,
 'x8 x10': 0.7496875,
 'x8 x11': 0.6844375,
 'x9 x10': 0.74575,
 'x9 x11

AUC ROC original features

X0 = slope_of_peak_exercise_st_segment = 0.6859999999999999

X1 = thal = 0.7745

X2 = chest_pain_type = 0.7696875

X3 = num_major_vessels = 0.7223125

X4 = fasting_blood_sugar_gt_120_mg_per_dl = 0.50125

X5 = resting_ekg_results = 0.5791250000000001

X6 = sex = 0.65625

X7 = exercise_induced_angina = 0.71

X8 = disc_resting_blood_pressure = 0.53675

X9 = disc_serum_cholesterol_mg_per_dl = 0.56575

X10 = disc_oldpeak_eq_st_depression = 0.72325

X11= disc_age = 0.615

X12 = disc_max_heart_rate_achieved = 0.716125



The worse predictors are:
X4 = fasting_blood_sugar_gt_120_mg_per_dl
X8 = disc_resting_blood_pressure
X9 = disc_serum_cholesterol_mg_per_dl
X11 = disc_age
Let's try to find poly features improving them.

After the analysis of poly features- the best combinations that improves the predective power of original features are:
'x4 x8': 0.5523750000000001
'X9 X11': 0.6375625
    

In [105]:
auc(newdf[['x4 x8', 'x9 x11']], y, clf)

0.711125

In [109]:
def prepareDataBasedOnAnalysis(data, mappingDict=None, km=None ):
    """
    use this for preparing data as an input for final data selection. 
    It discretize the features and calculates woe and addes kmlabel 
    """
    toDrop = ['patient_id', 'resting_blood_pressure', 'serum_cholesterol_mg_per_dl', 'oldpeak_eq_st_depression',
         'age', 'max_heart_rate_achieved', 'HR_adj_exerc_ind_ST_seg_depr']
    
    df = data.copy()
    # suggested feature from article
    df['HR_adj_exerc_ind_ST_seg_depr'] = df['oldpeak_eq_st_depression']/df['max_heart_rate_achieved']
    # discretization of features
    df['disc_resting_blood_pressure'] = df.apply(get_resting_blood_pressure_categories, axis=1)
    df['disc_serum_cholesterol_mg_per_dl'] = df.apply(get_serum_cholesterol_mg_per_dl_categ, axis=1)
    df['disc_oldpeak_eq_st_depression'] = df.apply(get_oldpeak_eq_st_depression_categ, axis=1)
    df['disc_age'] = df.apply(getAgeGroup, axis=1)
    df['disc_max_heart_rate_achieved'] = df.apply(get_max_heart_rate_achieved_categories, axis=1)
    df['disc_HR_adj_exerc_ind_ST_seg_depr'] = df.apply(get_disc_HR_adj_exerc_ind_ST_seg_categories, axis=1)
    df = df.drop(toDrop, axis=1)
    
    #making all fetures categoies
    for fea in df.columns:
        df[fea] = df[fea].astype('category')
    
    #when preparing training data creates a mappingDict from WoE for each feature
    if mappingDict is None:
        mappingDict = getWoe(df)
    
    # replaces the feature values with mapped WoE values
    for feat, attributes in mappingDict.items():
        df[feat].replace(attributes, inplace=True)
    
    # Clustering, creates clusering model from training data
    if km is None:
        df = df.drop('heart_disease_present', axis=1)
        km = KMeans(n_clusters=2)
        km.fit(df.drop(['disc_oldpeak_eq_st_depression'], axis=1))
        df['kmlabel'] = km.labels_      
    else:
        df['kmlabel'] = km.predict(df_mean.drop(['disc_oldpeak_eq_st_depression'], axis=1))
   
     #adding selected poly features
    df['fasting_blood_sugar_gt_120_mg_per_dl * disc_resting_blood_pressure'] = df.fasting_blood_sugar_gt_120_mg_per_dl *df.disc_resting_blood_pressure
    df['disc_serum_cholesterol_mg_per_dl * disc_age'] = df.disc_serum_cholesterol_mg_per_dl * df.disc_age
    return  df, mappingDict, km

In [110]:
df_for_fea_sel, mappingModel, km = prepareDataBasedOnAnalysis(df)

In [111]:
df_for_fea_sel.corr().style.background_gradient(cmap='viridis')

,slope_of_peak_exercise_st_segment,thal,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,sex,exercise_induced_angina,disc_resting_blood_pressure,disc_serum_cholesterol_mg_per_dl,disc_oldpeak_eq_st_depression,disc_age,disc_max_heart_rate_achieved,disc_HR_adj_exerc_ind_ST_seg_depr,kmlabel,fasting_blood_sugar_gt_120_mg_per_dl * disc_resting_blood_pressure,disc_serum_cholesterol_mg_per_dl * disc_age
slope_of_peak_exercise_st_segment,1,0.277372,0.253361,0.127485,0.00413826,0.171035,0.0539598,0.227293,0.0740446,0.00503714,0.514622,0.150916,0.441225,0.487936,0.377806,-0.0720306,-0.117226
thal,0.277372,1,0.386735,0.206087,0.000915133,-0.0512967,0.406214,0.400911,0.0649583,-0.0175396,0.335129,0.109513,0.27981,0.26623,0.74651,-0.0646838,-0.0844618
chest_pain_type,0.253361,0.386735,1,0.2899,-0.0924712,0.12041,0.180413,0.471657,0.0332561,0.0291714,0.235625,0.16621,0.404889,0.24023,0.641648,-0.0295906,-0.144294
num_major_vessels,0.127485,0.206087,0.2899,1,0.170377,0.0761733,0.0849622,0.170242,0.0355596,0.0982212,0.191221,0.297631,0.296338,0.184862,0.404091,-0.0463482,-0.311751
fasting_blood_sugar_gt_120_mg_per_dl,0.00413826,0.000915133,-0.0924712,0.170377,1,0.0414043,0.0660099,-0.00595575,0.120066,0.0327796,-0.040776,0.131587,-0.0634244,-0.0549615,-0.0429308,-0.170482,-0.121622
resting_ekg_results,0.171035,-0.0512967,0.12041,0.0761733,0.0414043,1,0.00982446,0.0694339,0.130998,0.136366,0.165572,0.142403,0.162091,0.150173,0.113938,-0.132109,-0.184973
sex,0.0539598,0.406214,0.180413,0.0849622,0.0660099,0.00982446,1,0.251096,0.0134994,-0.110556,0.110839,-0.0830081,0.0883773,0.120637,0.362749,-0.0166259,0.09751
exercise_induced_angina,0.227293,0.400911,0.471657,0.170242,-0.00595575,0.0694339,0.251096,1,0.0702533,0.0553779,0.267997,0.0891826,0.372317,0.292921,0.570082,-0.0692879,-0.0743426
disc_resting_blood_pressure,0.0740446,0.0649583,0.0332561,0.0355596,0.120066,0.130998,0.0134994,0.0702533,1,0.145313,0.173511,0.224288,0.0636638,0.173857,0.113853,-0.998605,-0.212816
disc_serum_cholesterol_mg_per_dl,0.00503714,-0.0175396,0.0291714,0.0982212,0.0327796,0.136366,-0.110556,0.0553779,0.145313,1,-0.0658712,0.137368,0.124934,-0.0377999,0.0333735,-0.146669,-0.535856


There is no pearson correlation higher than 0.75 

Let's try the best predictor with calculating the auc roc for each of them based on DTC

In [112]:
dfwork = pd.concat([df_for_fea_sel, df.heart_disease_present], axis=1)
def auc1(variables, target, basetable, clf):
    X = basetable[variables]
    y = basetable[target]
    clf.fit(X, y)
    predictions = clf.predict_proba(X)[:,1] 
    auc = roc_auc_score(y, predictions) 
    return(auc)

def next_best(current_variables,candidate_variables, target, basetable, clf): 
    best_auc = -1
    best_variable = None
    for v in candidate_variables:
        auc_v = auc1(current_variables + [v], target, basetable, clf)
        if auc_v >= best_auc: 
            best_auc = auc_v 
            best_variable = v
    return best_variable, best_auc


#Selecting feature with DecisionTreeClassifier
clf = DecisionTreeClassifier()
current_variables = []
candidate_variables = list(dfwork.drop('heart_disease_present', axis=1).columns)
target = "heart_disease_present"
max_number_variables = 15
number_iterations = min(max_number_variables, len(candidate_variables)) 

aucs_cum = [0]
for i in range(0,number_iterations):
    next_var, last_auc = next_best(current_variables,candidate_variables,target,dfwork, clf)
    if last_auc > aucs_cum[-1]:
        current_variables = current_variables + [next_var] 
        candidate_variables.remove(next_var)
        aucs_cum.append(last_auc)
current_variables

['kmlabel',
 'disc_serum_cholesterol_mg_per_dl * disc_age',
 'fasting_blood_sugar_gt_120_mg_per_dl * disc_resting_blood_pressure',
 'disc_max_heart_rate_achieved',
 'disc_oldpeak_eq_st_depression',
 'chest_pain_type',
 'num_major_vessels']